In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
# Mecab 인스톨
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4MB 146kB/s 
     |████████████████████████████████| 460kB 45.2MB/s 
     |████████████████████████████████| 92kB 14.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-05-16 06:05:29--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::22c2:513, ...
Con

In [ ]:
import torch

NLP = torch.FloatTensor([0,1,0,0,0,0,0,1])
learning = torch.FloatTensor([0,0,0,1,0,0,0,1])

print(torch.cosine_similarity(NLP, learning, dim=0))

tensor(0.5000)


In [ ]:
from konlpy.tag import Mecab

m = Mecab()
res = m.pos("기계학습을 위한 텍스트 처리 방법을 공부해보자.")
print(res)

[('기계', 'NNG'), ('학습', 'NNG'), ('을', 'JKO'), ('위한', 'VV+ETM'), ('텍스트', 'NNG'), ('처리', 'NNG'), ('방법', 'NNG'), ('을', 'JKO'), ('공부', 'NNG'), ('해', 'XSV+EC'), ('보', 'VX'), ('자', 'EF'), ('.', 'SF')]


In [ ]:
# 사전만들기 make_dic 함수수
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np

def make_dic(texts):

  words = []    # 단어 이곳에 저장
  m = Mecab()

  for snt in texts: # snt에 문장이 들어감
    res = m.pos(snt) # 형태소 분석, 품사가 무엇인지 부착
    # print(res)
    for (lex,pos) in res:
      if pos[0] == 'N' or pos[0] == 'V': #하늘 - N, 을 - J .. 약 45개의 품사, V는 동사, 시작이 N이면 체언
        words.append(lex)

  # 단어의 빈도수 계산
  dic = FreqDist(np.hstack(words))
  print('사전 크기: {0:d}'.format(len(dic)))

  # 인덱스를 저장할 변수 초기화
  indexes = {}
  words = {}

  # 단어에 고유 번호(인덱스) 부여
  for num, word in enumerate(dic): # dictionary에서 하나씩 가져오기 위해 enumerate() 사용용
    idx = num+2 # +2 중요함
    indexes[word]= idx # word를 인덱스로
    words[idx] = word #인덱스를 word로로

  # 문장 길이 정규화에 사용한 패딩 인덱스
  indexes['pad'] = 1 # 빈곳에 pad로 채움 -- 1번이 pad
  words[1] = 'pad' 
  # 사전에 없는 단어에 대한 인덱스
  indexes['unk'] = 0
  words[0] = 'unk'
 
  return (indexes, words)

def word2index(indexes, word):
  idx = indexes[word] if word in indexes else indexes['unk'] # index에 word값이 있으면 뽑아주고 없으면 unknown
  return idx

def index2word(words, index):
  w = words[index] if index in words else -1
  return w

# corpus
texts = ["죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를",
         "잎새에 이는 바람에도 나는 괴로워 했다.",
         "별을 노래하는 마음으로 모든 죽어 가는 것을 사랑해야지",
         "그리고 나한테 주어진 길을 걸어가야겠다.",
         "오늘 밤에도 별이 바람에 스치운다."]

(indexes, words) = make_dic(texts)
 
print(word2index(indexes,'하늘'), word2index(indexes,'학교')) # '학교' 단어는 없으므로 0으로 출력됨
print(index2word(words,4), index2word(words,0)) # unk는 없는 단어(out of vocabulary)

사전 크기: 24
4 0
하늘 unk


In [ ]:
# Word2Vec 사용하기 
# 학습 데이터 만들기
from konlpy.tag import Mecab
import numpy as np
from gensim.models import Word2Vec, KeyedVectors

texts = ["죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를",
         "잎새에 이는 바람에도 나는 괴로워 했다.",
         "별을 노래하는 마음으로 모든 죽어 가는 것을 사랑해야지",
         "그리고 나한테 주어진 길을 걸어가야겠다.",
         "오늘 밤에도 별이 바람에 스치운다."]
         #주어진 corpus에서 형태소 분석석

# Word2Vec 학습에 사용할 데이터 만들기

m = Mecab()
result = []

for sent in texts:
  tag = m.pos(sent)
  words = []
  for (lex,pos) in tag:
    if pos[0] == 'N': # 명사류(체언)만 추출
      words.append(lex)
  result.append(words)

print(result,"\n") #[['날', '하늘', '점', '부끄럼'], ['잎새', 바람, '나']] => [['별', '노래', '마음', ... ]

# Word2Vec 학습시키기
model = Word2Vec(sentences=result, size=10, window=1, min_count=1, workers=1, sg=0)
#size: embedding 차원 - 한단어를 10차원으로 , window: 컨텍스트 윈도우 크기, min_count: 단어 최소 빈도 수 제한 , workers: 학습을 위한 프로세스 수, sg : 0(CBOW), 1(Skip-gram)

# 값 읽어오기
print(model.wv['하늘']) #심볼을 숫자화 -> word embedding /wv : word to vector
# 유사한 단어 가져오기
print(model.wv.most_similar("하늘"),"\n") # cosine similarity해서 하늘과 유사한 다른 단어 뽑아옴

# Word2Vec 모델 저장하기
model.wv.save_word2vec_format('/gdrive/My Drive/colab/text_rep/test_w2v')

# Word2Vec 모델 로드하기
loaded_model = KeyedVectors.load_word2vec_format("/gdrive/My Drive/colab/text_rep/test_w2v")

# 값 읽어오기
print(loaded_model.wv['하늘'])
# 유사한 단어 가져오기
print(loaded_model.wv.most_similar("하늘"))

[['날', '하늘', '점', '부끄럼'], ['잎새', '바람', '나'], ['별', '노래', '마음', '것', '사랑'], ['나', '길'], ['밤', '별', '바람']] 

[-0.00181364  0.01349637  0.0494625  -0.04351401 -0.0074907  -0.00855643
  0.01056736 -0.04086038  0.0170731   0.01116189]
[('날', 0.2550535202026367), ('사랑', 0.19656619429588318), ('바람', 0.18432015180587769), ('노래', 0.14699342846870422), ('밤', -0.058490533381700516), ('마음', -0.08099609613418579), ('것', -0.0850621908903122), ('점', -0.15721188485622406), ('별', -0.16460390388965607), ('잎새', -0.26042240858078003)] 

[-0.00181364  0.01349637  0.0494625  -0.04351401 -0.0074907  -0.00855643
  0.01056736 -0.04086038  0.0170731   0.01116189]
[('날', 0.2550535202026367), ('사랑', 0.19656619429588318), ('바람', 0.18432015180587769), ('노래', 0.14699342846870422), ('밤', -0.058490533381700516), ('마음', -0.08099609613418579), ('것', -0.0850621908903122), ('점', -0.15721188485622406), ('별', -0.16460390388965607), ('잎새', -0.26042240858078003)]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [ ]:
# GloVe 사용하기
# GloVe 인스톨톨
# colab 환경에 glove 설치
! pip install glove-python-binary

     |████████████████████████████████| 952kB 29.2MB/s 


In [ ]:
from konlpy.tag import Mecab
import numpy as np
from glove import Corpus, Glove

texts = ["죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를",
         "잎새에 이는 바람에도 나는 괴로워 했다.",
         "별을 노래하는 마음으로 모든 죽어 가는 것을 사랑해야지",
         "그리고 나한테 주어진 길을 걸어가야겠다.",
         "오늘 밤에도 별이 바람에 스치운다."]

# GloVe 학습에 사용할 데이터 만들기

m = Mecab()
result = []

for sent in texts:
  tag = m.pos(sent)
  words = []
  for (lex,pos) in tag:
    if pos[0] == 'N':
      words.append(lex)
  result.append(words)

print(result,"\n")

# 학습 및 평가가
# Co-Occurrence Matrix 생성
corpus = Corpus() 
corpus.fit(result, window=1) #corpus에 값을 채움, window=1 한단어 떨어져있는 것까지만 카운팅팅

# GloVe 학습시키기
glove = Glove(no_components=10, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=1, verbose=True)
glove.add_dictionary(corpus.dictionary)

# 값 읽어오기
print(glove.word_vectors[glove.dictionary['하늘']]) # id를 넣으면 word_vector를 가져옴
# 유사한 단어 가져오기
print(glove.most_similar("하늘"),"\n")

# GloVe 모델 저장하기
glove.save('/gdrive/My Drive/colab/text_rep/test_glove')

# GloVe 모델 로드하기
loaded_model = glove.load("/gdrive/My Drive/colab/text_rep/test_glove")

# 값 읽어오기
print(loaded_model.word_vectors[glove.dictionary['하늘']]) # word2index와 같은 기능
# 유사한 단어 가져오기
print(loaded_model.most_similar("하늘"))

[['날', '하늘', '점', '부끄럼'], ['잎새', '바람', '나'], ['별', '노래', '마음', '것', '사랑'], ['나', '길'], ['밤', '별', '바람']] 

Performing 20 training epochs with 1 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
[ 0.02104036 -0.01404578 -0.04851466  0.01803728 -0.01523357 -0.04068209
 -0.04581227 -0.04701197 -0.00781063 -0.01010458]
[('부끄럼', 0.6554835514510583), ('점', 0.5460033054824257), ('나', 0.3306888587044728), ('날', 0.30147415271006095)] 

[ 0.02104036 -0.01404578 -0.04851466  0.01803728 -0.01523357 -0.04068209
 -0.04581227 -0.04701197 -0.00781063 -0.01010458]
[('부끄럼', 0.6554835514510583), ('점', 0.5460033054824257), ('나', 0.3306888587044728), ('날', 0.30147415271006095)]


In [ ]:
import torch
import torch.nn as nn

train_data = 'I like deep learning I like NLP I enjoy flying'

# 중복을 제거한 단어들의 집합인 단어 집합 생성.
word_set = set(train_data.split()) 

# 단어 집합의 각 단어에 고유한 정수 맵핑
vocab = {word: i+2 for i, word in enumerate(word_set)} # 미등록어와 패딩을 위한 인덱스 부여
vocab['unk'] = 0
vocab['pad'] = 1
print(vocab)

# 임베딩 테이블: (단어수, 임베딩 사이즈)
# Word2Vec, GloVe, Random
# 단어 수 만큼 lookup table을 만듬 - 세팅해서 tensor로 만듬
embedding_table = torch.FloatTensor([[ 0.0,  0.0,  0.0],
                                     [ 0.0,  0.0,  0.0],
                                     [ 0.1,  0.8,  0.3],
                                     [ 0.7,  0.8,  0.2],
                                     [ 0.1,  0.8,  0.7],
                                     [ 0.9,  0.2,  0.1],
                                     [ 0.1,  0.1,  0.9],
                                     [ 0.2,  0.1,  0.7],
                                     [ 0.3,  0.1,  0.1]])

# 입력 문장
input_snt = 'I like football'.split()

# 각 단어를 정수로 변환
idxes=[]

for word in input_snt:
  idx = vocab[word] if word in vocab else vocab['unk']
  idxes.append(idx)

idxes = torch.LongTensor(idxes) #인덱스를 tensor로 바꿈

# 입력 문장의임베딩가져오기: ['I', 'like', 'football (unk)']
lookup_result = embedding_table[idxes, :] # 3개의 단어 -> 인덱스로 가져옴
print(lookup_result)

# 임베딩 층 만들기
embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=3, padding_idx=1) #backpropagation하면서 업데이트
print(embedding_layer.weight)

{'enjoy': 2, 'like': 3, 'learning': 4, 'flying': 5, 'deep': 6, 'I': 7, 'NLP': 8, 'unk': 0, 'pad': 1}
tensor([[0.2000, 0.1000, 0.7000],
        [0.7000, 0.8000, 0.2000],
        [0.0000, 0.0000, 0.0000]])
Parameter containing:
tensor([[-0.1289, -0.8329, -0.6357],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.3707,  0.9090,  0.5469],
        [ 0.6152, -0.2019,  0.4137],
        [ 1.9100,  0.2001,  0.6432],
        [-1.5340,  0.2546, -0.1211],
        [-1.1414,  0.1044, -0.4543],
        [-1.6581,  0.0898,  1.7386],
        [-0.6676,  0.4591, -0.1105]], requires_grad=True)


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 6.7MB/s 


In [ ]:
import sentencepiece as spm

input_file = "/gdrive/MyDrive/colab/text_rep/corpus.txt"
vocab_size = 500
model_name = 'subword_kor'
model_type = 'bpe'
user_defined_symbols = '[PAD],[UNK],[CLS],[SEP],[MASK],[UNK1],[UNK2],[UNK3],[UNK4],[UNK5]'

input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --user_defined_symbols=%s --model_type=%s'
cmd = input_argument%(input_file, model_name, vocab_size,user_defined_symbols, model_type)

spm.SentencePieceTrainer.Train(cmd)

sp = spm.SentencePieceProcessor()
sp.Load("subword_kor.model")
wp = sp.EncodeAsPieces("하늘을 우러러")
id = sp.EncodeAsIds("하늘을 우러러")
print(wp)
print(id)

wp2snt= sp.DecodePieces(wp)
id2snt = sp.DecodeIds(id)
print(wp2snt)
print(id2snt)

print("Vocab. Size =",sp.GetPieceSize())
print(sp.IdToPiece(58))
print(sp.PieceToId('하'))

['▁', '하', '늘', '을', '▁우', '러', '러']
[174, 193, 356, 186, 80, 315, 315]
하늘을 우러러
하늘을 우러러
Vocab. Size = 500
▁강
193
